<a href="https://colab.research.google.com/github/darsh7807/learn-llm-with-me/blob/main/llms_from_scratch/ch2/tokenisation_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
import  urllib.request
url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
file_path="the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7bfa13301c70>)

In [94]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

print("Total num of chars: ", len(raw_text))
print(raw_text[:99])

Total num of chars:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [95]:
import re
text = "hello, world. This, is test."
result = re.split(r'(\s)', text)
print(result)
print(len(result))
result = re.split(r'([,.]\s)', text)
print(result)
print(len(result))

result = [item for item in result if item.strip()]
print(result)



['hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'test.']
9
['hello', ', ', 'world', '. ', 'This', ', ', 'is test.']
7
['hello', ', ', 'world', '. ', 'This', ', ', 'is test.']


In [96]:
text = "hello, world, Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
print(result)
result = [item.strip() for item in result if item.strip()]
print(result)



['hello', ',', '', ' ', 'world', ',', '', ' ', 'Is', ' ', 'this', '--', '', ' ', 'a', ' ', 'test', '?', '']
['hello', ',', 'world', ',', 'Is', 'this', '--', 'a', 'test', '?']


In [97]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [98]:
all_words = sorted(set(preprocessed))
vocab_size=len(all_words)
print(vocab_size)

1130


In [99]:
vocab = {token:integer  for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
  print(item)
  if i>5:
    break;

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)


In [100]:
class SimpleTokeniserV1:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str= {i:s for s, i in vocab.items()}

  def encode(self, text):
    preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    id =[self.str_to_int[i] for i in preprocessed]
    return id


  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    return text




In [101]:
tokeniser = SimpleTokeniserV1(vocab)
text = "the last he painted"

# ids = tokeniser.encode(text)
# print(len(ids))

In [102]:
class SimpleTokeniserV1:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str= {i:s for s, i in vocab.items()}

  def encode(self, text):
    # Corrected regex to match the one used for vocabulary creation
    preprocessed = re.split(r'([,.:;?_!"()\\]|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids =[self.str_to_int[i] for i in preprocessed]
    return ids


  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    return text

In [103]:
tokeniser = SimpleTokeniserV1(vocab)
text = "the last he painted"

ids = tokeniser.encode(text)
print(len(ids))

4


In [104]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
all_words = sorted(set(preprocessed))
all_words.extend(["<|endoftext|>","<|unk|>"])
vocab_size=len(all_words)
print(vocab_size)
vocab = {token:integer  for integer, token in enumerate(all_words)}

4690
1132


In [105]:
class SimpleTokeniserV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str= {i:s for s, i in vocab.items()}

  def encode(self, text):
    # Corrected regex to match the one used for vocabulary creation (using \' for single quote)
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    # Corrected the unknown token string from "|<unk>|" to "<|unk|>" and fixed typo 'itme' to 'item'
    preprocessed = [ item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
    ids =[self.str_to_int[i] for i in preprocessed]
    return ids


  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    return text

In [106]:
text1 = "hello, do you like tea?"
tokeniser = SimpleTokeniserV2(vocab)
print(tokeniser.encode(text1))

[1131, 5, 355, 1126, 628, 975, 10]


In [107]:
from importlib.metadata import version
import tiktoken

print(version("tiktoken"))

0.12.0


In [108]:
tokeniser = tiktoken.get_encoding("gpt2")
text = "hello, do you like tea?"
print(tokeniser.encode(text))

[31373, 11, 466, 345, 588, 8887, 30]


In [109]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokeniser, max_length, stride):
    self.input_ids = []
    self.target_ids = []
    token_ids = tokeniser.encode(txt)
    for i in range (0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [110]:
def createDataloader_V1(txt, batch_size=4, max_length=256, stride=256,
                        shuffle=True, drop_last=True,
                        num_workers=0
                        ):
  tokerniser = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokeniser, max_length, stride)
  dataloader = DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )
  return dataloader

In [113]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

dataloader = createDataloader_V1(raw_text,
                                 batch_size=1,
                                 max_length=4,
                                 stride=1,
                                 shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch=next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [121]:
dataloader = createDataloader_V1(raw_text,
                                 batch_size=8,
                                 max_length=4,
                                 stride=4,
                                 shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch[0])
print(first_batch[1])

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [122]:
torch.manual_seed(123)
output_dim=3
vocab_size=6
embedding_layer=torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)


Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [138]:
torch.manual_seed(123)
output_dim=256
vocab_size=50257
token_embedding_layer=torch.nn.Embedding(vocab_size, output_dim)
max_length=4
dataloader = createDataloader_V1(raw_text,
                                 batch_size=8,
                                 max_length=max_length,
                                 stride=max_length,
                                 shuffle=False)
data_iter = iter(dataloader)
input, targets = next(data_iter)
token_embedding = token_embedding_layer(input)
print(token_embedding.shape)



torch.Size([8, 4, 256])


In [139]:
context_length = max_length
pos_embedding_layer=torch.nn.Embedding(context_length, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
print(pos_embedding.shape)

torch.Size([4, 256])


In [141]:
input_embeddings = token_embedding + pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])
